Вам необходимо написать функцию, которая будет основана на поиске по сайту habr.com. Функция в качестве параметра должна принимать список запросов для поиска (например, ['python', 'анализ данных']) и на основе материалов, попавших в результаты поиска по каждому запросу, возвращать датафрейм вида:

<дата> - <заголовок> - <ссылка на материал>
В рамках задания предполагается работа только с одной (первой) страницей результатов поисковой выдачи для каждого запроса. Материалы в датафрейме не должны дублироваться, если они попадали в результаты поиска для нескольких запросов из списка.

https://habr.com/ru/search/?q=python&target_type=posts&order=relevance

In [97]:
import requests as res
from  bs4 import BeautifulSoup
import re
import pandas as pd
import time 

class Habr:
    def __init__(self,param):
        base_url = 'https://habr.com'
        search_url = base_url + '/ru/search/'
        all_links = []
        params = {  
            'q': param
        }
        headers = {
            'User-Agent': 'Mozilla/5.0'
        }

        r = res.get(search_url,params=params,headers=headers)
        time.sleep(0.5)
        Soup = BeautifulSoup(r.text,'html.parser')

        divs = Soup.find_all('div',class_='tm-article-snippet')
        for div in divs:
            links = div.find_all('a',class_='tm-article-snippet__readmore')
            for link in links:
                if link.get('href'):
                    all_links.append(link.get('href'))

        link_set = set(all_links)
        full_links = list(map(lambda x: base_url+x,link_set))

        self.full_links = full_links
    
    def search_habr(self):
        rows =[]
        for link in self.full_links:
            try:
                r = res.get(link)
                soup = BeautifulSoup(r.text,'html.parser')

                span_data = soup.find('span','tm-article-datetime-published')
                data_ = pd.to_datetime(span_data.find('time').get('datetime'))
                title = soup.find('h1','tm-title tm-title_h1').text

                row = {'date': data_,
                    'title':title,
                    'url':link}
                rows.append(row)
            except Exception as e:
                print(f'Ошибка при обработке {link}: {e}')
        data_news = pd.DataFrame(rows).reset_index(drop=True)
        return data_news

habr = Habr(param='python')
habr.search_habr()










,date,title,url
0,2024-07-11 06:57:03+00:00,Инсайдерам Microsoft 365 станет доступен редак...,https://habr.com/ru/news/828098/
1,2023-08-22 17:14:29+00:00,Microsoft добавила Python в Excel,https://habr.com/ru/news/756266/
2,2020-12-04 18:03:25+00:00,Python как компилируемый статически типизирова...,https://habr.com/ru/news/531402/
3,2022-01-20 15:37:16+00:00,Курс «Python для инженеров». Старт 3 потока 31...,https://habr.com/ru/companies/slurm/news/646825/
4,2024-07-01 11:46:51+00:00,Релиз открытой библиотеки для быстрой обработк...,https://habr.com/ru/news/825868/
5,2021-11-16 13:09:46+00:00,EPAM разработала бесплатный курс по программир...,https://habr.com/ru/companies/epam_systems/new...
6,2022-01-13 15:35:30+00:00,Открытый урок «Пишем Custom Prometheus Exporte...,https://habr.com/ru/companies/slurm/news/645485/
7,2023-07-05 13:18:58+00:00,"Разработчик выпустил инструмент, решающий проб...",https://habr.com/ru/news/746082/
8,2023-09-19 04:42:15+00:00,Microsoft представила дополнение Python Editor...,https://habr.com/ru/news/761862/
9,2023-11-09 11:05:15+00:00,Alchemer совместно с JetBrains и Python Softwa...,https://habr.com/ru/news/772872/


Функция из обязательной части задания должна быть расширена следующим образом:

кроме списка ключевых слов для поиска необходимо объявить параметр с количеством страниц поисковой выдачи. Т.е. при передаче в функцию аргумента 4 необходимо получить материалы с первых 4 страниц результатов;
в датафрейме должны быть столбцы с полным текстом найденных материалов и количеством лайков:
<дата> - <заголовок> - <ссылка на материал> - <текст материала> - <количество лайков>

In [144]:
import requests as res
from  bs4 import BeautifulSoup
import re
import pandas as pd
import time 

class Habr:
    def __init__(self,param,pages):
        all_full_links = []
        for page in range(1,pages+1):

            base_url = 'https://habr.com'
            search_url = base_url + f"/ru/search/page{page}"
            all_links = []
            params = {  
                'q': param
            }
            headers = {
                'User-Agent': 'Mozilla/5.0'
            }

            r = res.get(search_url,params=params,headers=headers)
            time.sleep(0.5)
            Soup = BeautifulSoup(r.text,'html.parser')

            divs = Soup.find_all('div',class_='tm-article-snippet')
            for div in divs:
                links = div.find_all('a',class_='tm-article-snippet__readmore')
                for link in links:
                    if link.get('href'):
                        all_links.append(link.get('href'))

            link_set = set(all_links)
            full_links = list(map(lambda x: base_url+x,link_set))
            all_full_links += full_links

        self.full_links = all_full_links
    
    def search_habr(self):
        rows =[]
        for link in self.full_links:
            try:
                r = res.get(link)
                soup = BeautifulSoup(r.text,'html.parser')

                span_data = soup.find('span','tm-article-datetime-published')
                data_ = pd.to_datetime(span_data.find('time').get('datetime'))
                title = soup.find('h1','tm-title tm-title_h1').text
                article_body = soup.find('div', class_='article-formatted-body article-formatted-body article-formatted-body_version-2')
                
                if not article_body:
                    article_body = soup.find('div', class_='article-formatted-body article-formatted-body article-formatted-body_version-1')
                if article_body:
                    paragraphs = article_body.find_all(['p', 'h2', 'li'])  # добавим заголовки и списки
                    text_post = ' '.join(p.get_text(strip=True) for p in paragraphs)
                else:
                   text_post = ''
                likes_div = soup.find('span', class_='tm-votes-meter__value tm-votes-meter__value_positive tm-votes-meter__value_appearance-article tm-votes-meter__value_rating tm-votes-meter__value')
                if likes_div:
                    likes = likes_div.get_text()
                    likes = 0
                row = {'date': data_,
                    'title':title,
                    'url':link,
                    'text':text_post,
                    'likes':likes}
                rows.append(row)
            except Exception as e:
                print(f'Ошибка при обработке {link}: {e}')
        data_news = pd.DataFrame(rows).reset_index(drop=True)
        return data_news

habr = Habr(param='python',pages=2)
habr.search_habr()


Ошибка при обработке https://habr.com/ru/news/828098/: empty separator
Ошибка при обработке https://habr.com/ru/news/695298/: empty separator
Ошибка при обработке https://habr.com/ru/news/756266/: empty separator
Ошибка при обработке https://habr.com/ru/news/531402/: empty separator
Ошибка при обработке https://habr.com/ru/companies/slurm/news/646825/: empty separator
Ошибка при обработке https://habr.com/ru/news/825868/: empty separator
Ошибка при обработке https://habr.com/ru/companies/slurm/news/645485/: empty separator
Ошибка при обработке https://habr.com/ru/news/761862/: empty separator
Ошибка при обработке https://habr.com/ru/news/772872/: empty separator
Ошибка при обработке https://habr.com/ru/news/498364/: empty separator
Ошибка при обработке https://habr.com/ru/companies/slurm/news/595093/: empty separator
Ошибка при обработке https://habr.com/ru/news/766124/: empty separator
Ошибка при обработке https://habr.com/ru/companies/yandex_praktikum/news/859368/: empty separator
Ош

,date,title,url,text,likes
0,2022-03-08 09:13:52+00:00,Вышел мартовский релиз расширения Python для V...,https://habr.com/ru/news/654707/,Вышел выпуск расширения Python для Visual Stud...,0
1,2023-11-21 07:27:21+00:00,"Исследование: ChatGPT работает лучше на Julia,...",https://habr.com/ru/news/775270/,Исследователь из Массачусетского технологическ...,0
2,2025-03-11 07:33:21+00:00,В библиотеке Python JSON Logger нашли уязвимос...,https://habr.com/ru/news/889810/,Исследователи безопасностинашлиуязвимость в би...,0
3,2023-03-06 12:53:17+00:00,Как стать Python-разработчиком: сессия вопросо...,https://habr.com/ru/companies/yandex_praktikum...,15 марта в 19:00 (Мск) Яндекс Практикум провед...,0


In [142]:
r = res.get('https://habr.com/ru/news/768702/')
soup = BeautifulSoup(r.text,'html.parser')
likes_div = soup.find('span', class_='tm-votes-meter__value tm-votes-meter__value_positive tm-votes-meter__value_appearance-article tm-votes-meter__value_rating tm-votes-meter__value')
if likes_div:
    likes = likes_div.get_text()
else:
    likes = 0
print(likes)

+13
